In [10]:
import math
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from collections import deque

In [17]:
class AI_trader:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=2000)
        self.inventory = []#input stocks
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95#discount
        self.epsilon = 1.0#randomness
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model("models/" + model_name) if is_eval else self._model()

    def model(self):#NN model for predictor
        model = keras.models.Sequential()
        model.add(keras.layers.Dense(64, input_dim=self.state_size, activation="relu"))
        model.add(keras.layers.Dense(32, activation="relu"))
        model.add(keras.layers.Dense(8, activation="relu"))
        model.add(keras.layers.Dense(3, activation="linear"))
        model.compile(loss="mse", optimizer='adam')

        return model

    def trade(self, state):#state: previous days for trade
        if not self.is_eval and np.random.rand() <= self.epsilon:#<epsilon: random sale
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def batch_train(self, batch_size):
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
  
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
    
        target = self.model.predict(state)
        print(target)
        target[0][action] = reward
  
        self.model.fit(state, target, epochs=1, verbose=1)#q value（reward）
  
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [12]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

In [13]:
def stocks_price_format(n):
    if n < 0:
        return "-${0:2f}".format(abs(n))
    else:
        return "${0:2f}".format(abs(n))

In [15]:
ibm = pd.read_csv('IBM.csv')
ibm.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,7.713333,7.713333,7.626667,7.626667,0.618153,387200
1,1962-01-03,7.626667,7.693333,7.626667,7.693333,0.623556,288000
2,1962-01-04,7.693333,7.693333,7.613333,7.616667,0.617343,256000
3,1962-01-05,7.606667,7.606667,7.453333,7.466667,0.605185,363200
4,1962-01-08,7.460000,7.460000,7.266667,7.326667,0.593837,544000
5,1962-01-09,7.360000,7.506667,7.360000,7.413333,0.600862,491200
6,1962-01-10,7.426667,7.460000,7.426667,7.426667,0.601942,299200
7,1962-01-11,7.446667,7.506667,7.446667,7.506667,0.608427,315200
8,1962-01-12,7.520000,7.573333,7.520000,7.520000,0.609508,435200
9,1962-01-15,7.546667,7.570000,7.546667,7.553333,0.612209,251200


In [16]:
data = ibm['Close'].to_numpy()

data = data[14000:]

In [7]:
def state_creator(data, timestep, window_size):
  
    starting_id = timestep - window_size + 1
#    print(data, timestep)
#    print(starting_id * [data[0]])
    
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = abs(starting_id) * [data[0]] + list(data[0:timestep+1])
    
    
#    print(starting_id)
    state = []
    
    for i in range(window_size - 1):
#        print(state)
#        print(windowed_data[i+1])
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
    return np.array([state])

In [8]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [11]:
trader = AI_trader(window_size)
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                704       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 27        
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for episode in range(1, episodes + 1):
  
    print("Episode: {}/{}".format(episode, episodes))
  
    state = state_creator(data, 0, window_size + 1)
  
    total_profit = 0
    trader.inventory = []
  
    for t in tqdm(range(data_samples)):
        action = trader.trade(state)
    
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0

        if action == 1: #Buying
            trader.inventory.append(data[t])
            print("Trader bought: ", stocks_price_format(data[t]))
      
        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)
      
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
        if t == data_samples - 1:
            done = True
        else:
            done = False

        
        trader.memory.append((state, action, reward, next_state, done))
    
        state = next_state
    
        if done:
            print("########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("########################")
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
        
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))

  0%|                                                                                          | 0/662 [00:00<?, ?it/s]

Episode: 1/1000
Trader bought:  $142.320007
Trader sold:  $142.070007  Profit: -$0.250000
Trader bought:  $142.139999
Trader bought:  $142.940002
Trader bought:  $143.740005
Trader sold:  $143.139999  Profit: $1.000000
Trader sold:  $144.080002  Profit: $1.140000
Trader sold:  $143.039993  Profit: -$0.700012
Trader bought:  $142.899994
Trader sold:  $142.449997  Profit: -$0.449997
Trader bought:  $145.990005
Trader bought:  $144.820007
Trader bought:  $144.550003
Trader bought:  $144.389999
Trader sold:  $145.880005  Profit: -$0.110000
Trader bought:  $145.259995
Trader bought:  $145.130005
Trader bought:  $146.559998
Trader sold:  $145.660004  Profit: $0.839997
Trader bought:  $145.660004
[[0.04506716 0.11310446 0.09202991]]
Train on 1 samples
1/1 [==============================] - 1s 524ms/sample - loss: 0.0021


  5%|████                                                                             | 33/662 [00:02<00:47, 13.34it/s]

Trader bought:  $145.080002
[[0.01509202 0.00668005 0.0126185 ]]
Train on 1 samples
1/1 [==============================] - 0s 7ms/sample - loss: 0.0021


  5%|████▏                                                                            | 34/662 [00:04<05:45,  1.82it/s]

...

 92%|█████████████████████████████████████████████████████████████████████████▋      | 610/662 [16:18<01:24,  1.63s/it]

[[0.62878865 0.9827683  1.0045353 ]]
Train on 1 samples
1/1 [==============================] - 0s 9ms/sample - loss: 0.0023


 92%|█████████████████████████████████████████████████████████████████████████▊      | 611/662 [16:19<01:23,  1.63s/it]

[[0.65510744 0.93069685 0.96653545]]
Train on 1 samples
1/1 [==============================] - 0s 8ms/sample - loss: 0.0027


 92%|█████████████████████████████████████████████████████████████████████████▉      | 612/662 [16:21<01:18,  1.57s/it]